## Using a SIFT based method for aligning image stacks. It uses the first  as a reference image to align the rest of the frames to. Read .tif or .tiff images stacks and saves the homography or transformation matrix.

#### I've used 16 bit tiff stacks, am not sure how I would change the method for reading files to accomodate for different tiff files

In [15]:
#apply sift on tiff files

#USING SIFT FOR ALIGNMENT
#https://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/

import cv2
import numpy as np
from skimage import io, color
import glob
from skimage.external.tifffile import TiffWriter, imsave
import os
 
MIN_MATCH_COUNT = 10
 
 
def alignImages(img1, img2): #function to align the images using SIFT
    
    #Convert 16 bit to 8 bit
    im1Gray=(img1/256).astype('uint8')
    im2Gray=(img2/256).astype('uint8')
    
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=1000,nOctaveLayers=3,sigma=1.60)
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(im1Gray,None)
    kp2, des2 = sift.detectAndCompute(im2Gray,None)

    '''
    test code for using FLANN based matcher
    #FLANN_INDEX_KDTREE = 0
    #index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    #search_params = dict(checks = 50)
    #flann = cv2.FlannBasedMatcher(index_params, search_params)

    #matches = flann.knnMatch(des1,des2,k=2)'''
   
    #create BFMatcher with default params (Brute-Force matching with SIFT)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)
    
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        h,w = img1.shape
        img3 = cv2.warpPerspective(img1,M, (w, h))

    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        matchesMask = None
        img3=img1 #no transformation, just return the same image without transformation (WARNING: this should be used only for blank frames)
        M=np.zeros((3,3)) #generate an array of zeros 
    return img3,M

folder=input("Enter the folder location")
print(os.listdir(folder))

all_zeros=0
count=0 #counter for the loop to check if homography is zero (used for blank frames)

for name in glob.glob(folder+"\\*"): # read all files in the folder 
    calcium_aligned=[] #store all the aligned frames
    homography=[] #store homography matrix

    calcium=[] #store the unaligned frames
    if name.endswith("tif" or "tiff"):
        im=name
        calcium=cv2.imreadmulti(name,flags=cv2.IMREAD_ANYDEPTH)
        #same parameters as imread
        #index 1 or calcium[1] will contain all the frames; len(calcium[1]) will return the no of frames
        #calcium[1][n] is the nth frame

        
        if(len(calcium[1])<10): #check if less than 10 frames
            print(name+" is less than 10 frames, not aligning.")
        else:
            print(name)
            imReference = calcium[1][0] #first image is used as reference template;
            calcium_aligned.append(imReference) #append the first frame to calcium_aligned stack
            new_frame_len=len(calcium[1])
            for i in (range(1,new_frame_len,1)):
                print("Image to align : ", (i+1))  
                print("Aligning images ...")
                im=calcium[1][i]
                #plt.imshow(im)
                # Registered image will be restored in imReg. 
                # The estimated homography will be stored in h. 
                imReg, h = alignImages(im, imReference)
                #imReference=imReg #align the image to the imReg or aligned image
                calcium_aligned.append(imReg)
                homography.append(h)
                all_zeros = not h.any() #check if homography matrix is zeros
                if(all_zeros): #if zeros, then use im as reference image
                    if(count==0): #if counter is zero (first time a blank frame), then assign reference image to orig this makes sure that even consecutive frames are blank, atleast the reference frame is still the original frame that had an image
                        orig=imReference #if blank frame or image, use the 'good' reference image for next loop
                    imReference=orig
                    count+=1
                else: #if homography matrix is good, then
                    imReference=imReg
                    count=0
                # Print estimated homography
                #print("Estimated homography : \n",  h)
                 
        # Write aligned image to disk.
        calcium_aligned=np.array(calcium_aligned) #convert to array instead of a list
        name=os.path.basename(name) #get file name
        outFilename = folder+"\\"+name+"_aligned_SIFT.tif"
        print("Saving aligned image : ", outFilename) 
        imsave(outFilename,calcium_aligned)
        print("Saving transformation matrix")
        outFilename = folder+"\\"+name+"_homography.txt"
        homography1=np.array(homography)
        homography1.tofile(outFilename,sep=",",format='%-0.7f') #it saves as a string

Enter the folder locationE:\Multiphoton\WT and A53T
['180208 A53T1A', '180208 A53T_1A_GSK_f41 MRS_f328 NK2_f370.lif - Series005_aligned.tif']
E:\Multiphoton\WT and A53T\180208 A53T_1A_GSK_f41 MRS_f328 NK2_f370.lif - Series005_aligned.tif
Image to align :  2
Aligning images ...
Image to align :  3
Aligning images ...
Image to align :  4
Aligning images ...
Image to align :  5
Aligning images ...
Image to align :  6
Aligning images ...
Image to align :  7
Aligning images ...
Image to align :  8
Aligning images ...
Image to align :  9
Aligning images ...
Image to align :  10
Aligning images ...
Image to align :  11
Aligning images ...
Image to align :  12
Aligning images ...
Image to align :  13
Aligning images ...
Image to align :  14
Aligning images ...
Image to align :  15
Aligning images ...
Image to align :  16
Aligning images ...
Image to align :  17
Aligning images ...
Image to align :  18
Aligning images ...
Image to align :  19
Aligning images ...
Image to align :  20
Aligning im

Image to align :  194
Aligning images ...
Image to align :  195
Aligning images ...
Image to align :  196
Aligning images ...
Image to align :  197
Aligning images ...
Image to align :  198
Aligning images ...
Image to align :  199
Aligning images ...
Image to align :  200
Aligning images ...
Image to align :  201
Aligning images ...
Image to align :  202
Aligning images ...
Image to align :  203
Aligning images ...
Image to align :  204
Aligning images ...
Image to align :  205
Aligning images ...
Image to align :  206
Aligning images ...
Image to align :  207
Aligning images ...
Image to align :  208
Aligning images ...
Image to align :  209
Aligning images ...
Image to align :  210
Aligning images ...
Image to align :  211
Aligning images ...
Image to align :  212
Aligning images ...
Image to align :  213
Aligning images ...
Image to align :  214
Aligning images ...
Image to align :  215
Aligning images ...
Image to align :  216
Aligning images ...
Image to align :  217
Aligning ima

Image to align :  390
Aligning images ...
Image to align :  391
Aligning images ...
Image to align :  392
Aligning images ...
Image to align :  393
Aligning images ...
Image to align :  394
Aligning images ...
Image to align :  395
Aligning images ...
Image to align :  396
Aligning images ...
Image to align :  397
Aligning images ...
Image to align :  398
Aligning images ...
Image to align :  399
Aligning images ...
Image to align :  400
Aligning images ...
Image to align :  401
Aligning images ...
Image to align :  402
Aligning images ...
Image to align :  403
Aligning images ...
Image to align :  404
Aligning images ...
Image to align :  405
Aligning images ...
Image to align :  406
Aligning images ...
Image to align :  407
Aligning images ...
Image to align :  408
Aligning images ...
Image to align :  409
Aligning images ...
Image to align :  410
Aligning images ...
Image to align :  411
Aligning images ...
Image to align :  412
Aligning images ...
Image to align :  413
Aligning ima

## Below code is to read the Leica .liff files and get an image series with more than 10 frames and perform alignment. Need to use Bioformats to open the .liff files and hence the  Javabridge (This took a while to figure out how to setup in Windows)

In [21]:
#code to read in  lif file (LEICA) and apply sift on green channel if its a multi channel image. 
#if java virtual machine error pops up, may need to restart kernel

#make the system is running 64 bit python
import platform
a=platform.architecture()
if a[0] == '32bit':
    print("The program may run out of memory as your python version is 32 bit, please use 64 bit")
elif a[0] == '64bit':
    print('Running 64 bit python')

#if java path not working, manually set it here
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11.0.2"


#read bioformats image
import javabridge as jv, bioformats as bf 
#if pip doesnt work, install javabridge separately (download appropriate (32/64bit wheel or.whl)))

jv.start_vm(class_path=bf.JARS)#, max_heap_size='1024000m') #start java virtual machine

#USING SIFT FOR ALIGNMENT
#https://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/

import cv2
import numpy as np
from skimage import io, color
import glob
from skimage.external.tifffile import TiffWriter, imsave
from skimage import img_as_uint
import warnings

#the 16 bit conversion:img_as_uint throws a warning about potential decrease in quality
warnings.filterwarnings('ignore', category=UserWarning, append=True) 

MIN_MATCH_COUNT = 10
 
def alignImages(img1, img2): #function to align the images using SIFT
    
    #Convert 16 bit to 8 bit
    im1Gray=(img1/256).astype('uint8')
    im2Gray=(img2/256).astype('uint8')
    
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=1000,nOctaveLayers=3,sigma=1.60)
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(im1Gray,None)
    kp2, des2 = sift.detectAndCompute(im2Gray,None)

    #test code for using FLANN based matcher
    #FLANN_INDEX_KDTREE = 0
    #index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    #search_params = dict(checks = 50)
    #flann = cv2.FlannBasedMatcher(index_params, search_params)

    #matches = flann.knnMatch(des1,des2,k=2)
   
    #create BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)
    
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        h,w = img1.shape
        img3 = cv2.warpPerspective(img1,M, (w, h))

    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        matchesMask = None
        img3=img1 #no transformation, just return same image (WARNING: this is only for blank frames)
        M=np.zeros((3,3)) #generate an array of zeros
    return img3,M

name=input("Select the liff file")
#set the location of the liff file
a=bf.ImageReader(name, perform_init=True)
reader=a.rdr

total_images=reader.getSeriesCount() #get the total no of images in the lif


image=[] #to store the no of images in the lif over 10 frames
duration=[] #store the no of frames in each tiff stack
channel=[] #store no of channels
calcium_aligned=[] #store all the aligned frames
homography=[] #store homography matrix
calcium=[] #store the unaligned frames

#loop to get no of series, no of frames and channels from the tif
for series in (range(0,(total_images),1)):
    reader.setSeries(series)
    frames=reader.getSizeT()
    ch=reader.getSizeC()
    if frames>10:  # if frames are greater than 10, hen append it to image; it will append all the images with greater than 10 frames
        image.append(series)
        duration.append(frames)
        channel.append(ch)
print(image, duration,channel)

#set the output folder
folder=os.path.dirname(name)
#if not os.path.exists(folder):
    #os.makedirs(folder)

for idx,i in enumerate(image):
    imReference = a.read(c=0,t=0,series=i,rescale=False) #first image is stored in imReference
    imReference=img_as_uint(imReference)#convert to 16 bit
    calcium_aligned.append(imReference)

    for t in (range(1,duration[idx],1)):
        print("Image to align : ", (t))  
        print("Aligning images ...")
        im=(a.read(c=0,t=t,series=i,rescale=False))
        im=img_as_uint(im) #convert to 16 bit
        #plt.imshow(im)
        # Registered image will be restored in imReg. 
        # The estimated homography will be stored in h. 
        imReg, h = alignImages(im, imReference)
        imReference=imReg #align the image to the imReg or previously aligned image
        calcium_aligned.append(imReg)
        homography.append(h)
        # Print estimated homography
        print("Estimated homography : \n",  h)

        #IF MULTIPLE IMAGES, THEN ADJUST INDENTS ACCORDINGLY
# Write aligned image to disk.
calcium_aligned=np.array(calcium_aligned) #convert to array instead of a list
name=os.path.basename(name) #get file name
outFilename = folder+"\\"+name+"_aligned_SIFT.tif"
print("Saving aligned image : ", outFilename) 
imsave(outFilename,calcium_aligned)
print("Saving transformation matrix")
outFilename = folder+"\\"+name+"_homography.txt"
homography1=np.array(homography)
homography1.tofile(outFilename,sep=",",format='%-0.7f') #it saves as a string

jv.kill_vm() #kill virtual machine

Running 64 bit python
Select the liff fileE:\test\180809 MP1 VEH GSK_f82 NKA_f439 ATP_f508.lif
[0] [691] [1]
Image to align :  1
Aligning images ...
Estimated homography : 
 [[ 9.99744599e-01 -2.37565942e-04  1.05365642e-01]
 [ 6.67280539e-05  9.99338039e-01  1.27260662e-01]
 [ 1.60415166e-07 -7.04664302e-07  1.00000000e+00]]
Image to align :  2
Aligning images ...
Estimated homography : 
 [[ 1.00085643e+00 -2.36959964e-04 -1.62794567e-01]
 [ 1.31406500e-03  9.99187608e-01 -1.31968602e-01]
 [ 1.34949443e-06 -8.61668214e-07  1.00000000e+00]]
Image to align :  3
Aligning images ...
Estimated homography : 
 [[ 9.99904603e-01 -1.47198325e-04  1.09612895e-01]
 [ 5.09766088e-04  9.99492086e-01 -1.81543807e-02]
 [ 5.71100658e-08 -2.40965712e-07  1.00000000e+00]]
Image to align :  4
Aligning images ...
Estimated homography : 
 [[ 9.99989033e-01 -3.23358723e-04  1.53527654e-01]
 [ 5.90595828e-04  9.99427624e-01 -6.28462099e-02]
 [ 3.38028277e-08 -3.12764132e-07  1.00000000e+00]]
Image to align 

Estimated homography : 
 [[ 9.96774271e-01  9.32047196e-05  1.17595533e+00]
 [-2.98450915e-03  9.98994911e-01  8.59169728e-01]
 [-3.69905127e-06  9.24861133e-07  1.00000000e+00]]
Image to align :  39
Aligning images ...
Estimated homography : 
 [[ 9.97700845e-01  2.65551607e-04  8.39170115e-01]
 [-2.81714223e-03  9.99881305e-01  5.50794570e-01]
 [-2.91505099e-06  1.04289448e-06  1.00000000e+00]]
Image to align :  40
Aligning images ...
Estimated homography : 
 [[ 9.97462535e-01 -6.69585147e-05  9.71564645e-01]
 [-2.46126987e-03  9.98832066e-01  6.73258310e-01]
 [-2.96943495e-06  4.18775191e-07  1.00000000e+00]]
Image to align :  41
Aligning images ...
Estimated homography : 
 [[ 9.98222349e-01 -2.46181615e-04  7.89824057e-01]
 [-1.58265186e-03  9.98546626e-01  4.82103662e-01]
 [-2.05718968e-06 -1.23677639e-08  1.00000000e+00]]
Image to align :  42
Aligning images ...
Estimated homography : 
 [[ 9.98114771e-01 -2.14183919e-04  8.52056702e-01]
 [-1.83066738e-03  9.98476969e-01  5.8940329

Estimated homography : 
 [[ 9.97835507e-01 -6.70793806e-04  1.57428405e+00]
 [ 1.35224792e-05  9.98049454e-01 -3.10673539e-01]
 [ 8.36181827e-08 -5.56260811e-07  1.00000000e+00]]
Image to align :  77
Aligning images ...
Estimated homography : 
 [[ 9.98113764e-01 -9.81411087e-04  1.55543678e+00]
 [ 7.49906188e-04  9.97648820e-01 -4.61709644e-01]
 [ 6.23025535e-07 -1.21998403e-06  1.00000000e+00]]
Image to align :  78
Aligning images ...
Estimated homography : 
 [[ 9.98545752e-01 -5.34034496e-04  1.37729006e+00]
 [ 1.03706897e-03  9.98390035e-01 -8.07984620e-01]
 [ 8.12308137e-07 -6.67729789e-07  1.00000000e+00]]
Image to align :  79
Aligning images ...
Estimated homography : 
 [[ 9.99191829e-01 -1.45635768e-03  1.68909772e+00]
 [ 6.54724801e-04  9.98809009e-01 -7.10490855e-01]
 [ 8.82170733e-07 -2.40686732e-08  1.00000000e+00]]
Image to align :  80
Aligning images ...
Estimated homography : 
 [[ 9.97380244e-01 -1.08119395e-03  1.43678925e+00]
 [-3.53121296e-04  9.98007997e-01 -4.8049783

Estimated homography : 
 [[ 1.00064382e+00 -9.04985558e-04  5.67621629e-01]
 [ 4.40771524e-04  9.95724122e-01  2.53615338e-01]
 [ 2.69259292e-06 -2.48922701e-06  1.00000000e+00]]
Image to align :  115
Aligning images ...
Estimated homography : 
 [[ 1.00047864e+00 -8.23287194e-04  5.86685555e-01]
 [ 2.94168360e-04  9.95684721e-01  3.06784128e-01]
 [ 2.45806504e-06 -2.35864774e-06  1.00000000e+00]]
Image to align :  116
Aligning images ...
Estimated homography : 
 [[ 1.00100546e+00 -8.48155094e-04  5.49031736e-01]
 [ 6.26124560e-04  9.95675584e-01  2.63088292e-01]
 [ 2.90038950e-06 -2.37307127e-06  1.00000000e+00]]
Image to align :  117
Aligning images ...
Estimated homography : 
 [[ 1.00143822e+00 -7.25090700e-04  4.82314485e-01]
 [ 9.76319131e-04  9.96291779e-01 -6.38425705e-02]
 [ 3.21563187e-06 -1.96679831e-06  1.00000000e+00]]
Image to align :  118
Aligning images ...
Estimated homography : 
 [[ 1.00088797e+00 -1.06144867e-03  6.72262634e-01]
 [ 9.43885223e-04  9.95495858e-01  8.643

Estimated homography : 
 [[ 1.00411181e+00  1.02642743e-03 -7.00250358e-01]
 [ 2.21598433e-03  1.00020160e+00 -2.16009001e+00]
 [ 4.96568712e-06  1.00787385e-06  1.00000000e+00]]
Image to align :  153
Aligning images ...
Estimated homography : 
 [[ 1.00543450e+00  1.63488414e-03 -1.20305977e+00]
 [ 2.84573324e-03  1.00220005e+00 -3.02990315e+00]
 [ 5.76231332e-06  2.00591349e-06  1.00000000e+00]]
Image to align :  154
Aligning images ...
Estimated homography : 
 [[ 1.00596902e+00  1.50227770e-03 -1.21308374e+00]
 [ 3.35926167e-03  1.00165895e+00 -2.85537827e+00]
 [ 6.40831121e-06  1.81664482e-06  1.00000000e+00]]
Image to align :  155
Aligning images ...
Estimated homography : 
 [[ 1.00624674e+00  8.15747811e-04 -1.11815731e+00]
 [ 4.03575976e-03  1.00058352e+00 -2.82465058e+00]
 [ 7.44204650e-06  3.80097042e-07  1.00000000e+00]]
Image to align :  156
Aligning images ...
Estimated homography : 
 [[ 1.00659038e+00  9.69709339e-04 -1.25168975e+00]
 [ 3.87244531e-03  1.00105314e+00 -2.967

...

Estimated homography : 
 [[ 1.00544032e+00 -5.57181735e-03  1.72683125e+00]
 [ 1.77783440e-04  9.93477005e-01 -2.97692061e+00]
 [ 1.49460436e-05 -8.38681045e-06  1.00000000e+00]]
Image to align :  533
Aligning images ...
Estimated homography : 
 [[ 1.00467178e+00 -5.84156674e-03  1.96135275e+00]
 [-2.69478251e-04  9.92907820e-01 -2.73564257e+00]
 [ 1.43329299e-05 -8.78155343e-06  1.00000000e+00]]
Image to align :  534
Aligning images ...
Estimated homography : 
 [[ 1.00531021e+00 -5.92858144e-03  1.88031084e+00]
 [-4.04269879e-05  9.93260941e-01 -2.89537071e+00]
 [ 1.48602335e-05 -8.72116176e-06  1.00000000e+00]]
Image to align :  535
Aligning images ...
Estimated homography : 
 [[ 1.00588485e+00 -5.87960512e-03  1.75380737e+00]
 [-7.77267110e-05  9.93896320e-01 -3.02061343e+00]
 [ 1.53080093e-05 -8.52217227e-06  1.00000000e+00]]
Image to align :  536
Aligning images ...
Estimated homography : 
 [[ 1.00605300e+00 -5.78157644e-03  1.70333596e+00]
 [ 2.79493232e-04  9.94141747e-01 -3.188

Estimated homography : 
 [[ 1.01082644e+00 -4.03029228e-03 -5.27611958e-01]
 [ 1.61702763e-03  9.98840358e-01 -5.49638289e+00]
 [ 1.73634822e-05 -6.47297438e-06  1.00000000e+00]]
Image to align :  571
Aligning images ...
Estimated homography : 
 [[ 1.01036563e+00 -3.87614336e-03 -4.74557395e-01]
 [ 1.24943590e-03  9.98817104e-01 -5.34214651e+00]
 [ 1.67472893e-05 -6.14094959e-06  1.00000000e+00]]
Image to align :  572
Aligning images ...
Estimated homography : 
 [[ 1.00988129e+00 -4.17963749e-03 -2.68508014e-01]
 [ 1.02342318e-03  9.98083415e-01 -5.08878085e+00]
 [ 1.66253177e-05 -6.75001620e-06  1.00000000e+00]]
Image to align :  573
Aligning images ...
Estimated homography : 
 [[ 1.00977008e+00 -4.60180182e-03 -6.71528815e-02]
 [ 1.27949728e-03  9.97807973e-01 -5.15465159e+00]
 [ 1.70191818e-05 -7.35435073e-06  1.00000000e+00]]
Image to align :  574
Aligning images ...
Estimated homography : 
 [[ 1.01033396e+00 -4.22617996e-03 -3.53045828e-01]
 [ 9.41440104e-04  9.98647628e-01 -5.139

Estimated homography : 
 [[ 1.00411704e+00 -6.01032939e-03  2.07074651e+00]
 [ 3.74222977e-04  9.92309350e-01 -3.92845089e+00]
 [ 1.32866230e-05 -9.67123245e-06  1.00000000e+00]]
Image to align :  609
Aligning images ...
Estimated homography : 
 [[ 1.00077751e+00 -7.78691082e-03  3.37808981e+00]
 [-1.42222974e-03  9.88297722e-01 -2.34875420e+00]
 [ 1.12110660e-05 -1.22687039e-05  1.00000000e+00]]
Image to align :  610
Aligning images ...
Estimated homography : 
 [[ 1.00060613e+00 -8.05047068e-03  3.49972959e+00]
 [-1.18392301e-03  9.87664501e-01 -2.15668703e+00]
 [ 1.12208517e-05 -1.25674727e-05  1.00000000e+00]]
Image to align :  611
Aligning images ...
Estimated homography : 
 [[ 1.00070869e+00 -8.15553410e-03  3.44778105e+00]
 [-1.21284466e-03  9.86896284e-01 -1.92451446e+00]
 [ 1.14977328e-05 -1.30087963e-05  1.00000000e+00]]
Image to align :  612
Aligning images ...
Estimated homography : 
 [[ 1.00241923e+00 -7.96960992e-03  2.93340552e+00]
 [ 2.11006170e-04  9.87949150e-01 -2.657

Estimated homography : 
 [[ 9.97219563e-01 -6.52220709e-03  3.36343560e+00]
 [-1.43871893e-03  9.86420730e-01 -2.01052582e+00]
 [ 7.79339557e-06 -1.11979521e-05  1.00000000e+00]]
Image to align :  647
Aligning images ...
Estimated homography : 
 [[ 9.96937162e-01 -6.21108683e-03  3.36620379e+00]
 [-1.61116170e-03  9.86667874e-01 -1.94992739e+00]
 [ 7.41333228e-06 -1.07306772e-05  1.00000000e+00]]
Image to align :  648
Aligning images ...
Estimated homography : 
 [[ 9.96850053e-01 -5.96056012e-03  3.34727718e+00]
 [-1.83758303e-03  9.86835881e-01 -1.79872919e+00]
 [ 7.02496963e-06 -9.98063064e-06  1.00000000e+00]]
Image to align :  649
Aligning images ...
Estimated homography : 
 [[ 9.95905141e-01 -6.13682864e-03  3.67008292e+00]
 [-2.37722238e-03  9.86350372e-01 -1.53288693e+00]
 [ 6.30323905e-06 -1.00855186e-05  1.00000000e+00]]
Image to align :  650
Aligning images ...
Estimated homography : 
 [[ 9.96649394e-01 -6.09609975e-03  3.43679588e+00]
 [-1.67473267e-03  9.86753470e-01 -1.960

Estimated homography : 
 [[ 9.91740477e-01 -5.37019060e-03  3.94848592e+00]
 [-4.23137117e-03  9.84738463e-01 -1.50206732e-01]
 [ 3.15166638e-06 -1.03767095e-05  1.00000000e+00]]
Image to align :  685
Aligning images ...
Estimated homography : 
 [[ 9.90948078e-01 -5.35317383e-03  4.07211171e+00]
 [-4.65968551e-03  9.84347390e-01  1.05643890e-01]
 [ 2.40942754e-06 -1.04604408e-05  1.00000000e+00]]
Image to align :  686
Aligning images ...
Estimated homography : 
 [[ 9.89280085e-01 -4.76691632e-03  4.32794808e+00]
 [-5.52121295e-03  9.84339777e-01  4.23137424e-01]
 [ 4.46322370e-07 -9.36828930e-06  1.00000000e+00]]
Image to align :  687
Aligning images ...
Estimated homography : 
 [[ 9.90246757e-01 -3.92979453e-03  3.88617083e+00]
 [-5.81033699e-03  9.86450522e-01 -1.09350635e-01]
 [ 7.21464185e-07 -8.05502255e-06  1.00000000e+00]]
Image to align :  688
Aligning images ...
Estimated homography : 
 [[ 9.90387692e-01 -4.12643563e-03  3.87345390e+00]
 [-5.57436197e-03  9.86186059e-01 -9.458

### I've written some code to read a folder full of tiff or liff files and process them individually. Just some extra for loops.